# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
results_dir = 'results/final-frontier/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

/Users/bencottier/Projects/training_cost_2/training-cost-trends/data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontier_pcd_df.loc[:, 'Training compute (FLOP)'] = pd.to_numeric(


In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(138, 5508, 73)

# Cost estimation

In [9]:
cost_df = estimate_costs(frontier_pcd_df, hardware_df, price_df)

==== System: Gemini Ultra ====
Trying Google TPU v4 at 2023-06-29 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Found price: 1.45 at 2023-06-01 00:00:00
Difference between purchase time and price date: 28 days 00:00:00 

==== System: Qwen-72B ====
Training time is required but no value found

==== System: Inflection-2 ====
Training time is required but no value found

==== System: Nemotron-3-8B ====
Trying NVIDIA A100 at 2023-08-28 00:00:00
Trying Amazon Web Services, Price per chip-hour (3-year CUD)
Found price: 1.45 at 2022-12-09 00:00:00
Difference between purchase time and price date: 262 days 00:00:00 

==== System: CogVLM ====
Training time is required but no value found

==== System: Yi-34B ====
Training time is required but no value found

==== System: Skywork-13B ====
Trying NVIDIA A800 at 2023-07-22 20:00:00
Trying Amazon Web Services, Price per chip-hour (3-year CUD)
Could not find price

Trying Microsoft Azure, Price per chip-hour (3-year CUD)
Could not fin

In [10]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Training data center,Archived links,Batch size,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training time (chip hours),Cost
36,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",NaN,https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,132000000.0,191400000.0
43,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license,https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,NaN,NaN,4000000.0,Table 1 https://arxiv.org/abs/2309.16609\n(thi...,Industry,NaN,NaN,NaN,NaN,NaN
51,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API accessible,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,NaN,NaN,NaN,Industry,checked,NaN,NaN,NaN,NaN
55,Nemotron-3-8B,Language,"Chat,Language Generation",NaN,SOTA improvement,"""The Nemotron-3-8B-QA model offers state-of-th...",Permissive license,https://developer.nvidia.com/blog/nvidia-ai-fo...,NaN,NVIDIA AI Foundation Models: Build Custom Ente...,...,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN
72,CogVLM,"Multimodal,Vision,Language","Image captioning,Visual question answering,Chat","Weihan Wang, Qingsong Lv, Wenmeng Yu, Wenyi Ho...",SOTA improvement,"""CogVLM-17B\nachieves state-of-the-art perform...",Permissive license,https://arxiv.org/abs/2311.03079\nhttps://hugg...,43.0,CogVLM: Visual Expert for Pretrained Language ...,...,NaN,NaN,NaN,NaN,"Academia,Industry,Academia",checked,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,6105.0,Google's Neural Machine Translation System: Br...,...,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,414720.0,178329.6
1126,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN
1130,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,154061.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN
1132,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN


In [11]:
cost_df['Cost'].notna().sum()

36

Use the below to check data availability for specific systems

In [13]:
# system = 'AlphaGo Fan'
# print('Cost:', cost_df.loc[system, 'Cost'])
# print('Training hardware:', cost_df.loc[system, 'Training hardware'])
# print('Training time (hours):', cost_df.loc[system, 'Training time (hours)'])
# print('Hardware quantity:', cost_df.loc[system, 'Hardware quantity'])
# print('Hardware utilization:', cost_df.loc[system, 'Hardware utilization'])

# Apply inflation adjustment

In [14]:
# TODO move to function

In [15]:
cost_df['Publication date']

36     2023-12-06
43     2023-11-30
51     2023-11-22
55     2023-11-15
72     2023-11-06
          ...    
1090   2016-09-26
1126   2016-01-27
1130   2015-12-10
1132   2015-12-08
1136   2015-10-01
Name: Publication date, Length: 138, dtype: datetime64[ns]

In [16]:
from_year_month = cost_df['Publication date'].apply(str)
cost_df['Publication date'] = from_year_month

In [17]:
cost_df['Publication date']

36      2023-12-06 00:00:00
43      2023-11-30 00:00:00
51      2023-11-22 00:00:00
55      2023-11-15 00:00:00
72      2023-11-06 00:00:00
               ...         
1090    2016-09-26 00:00:00
1126    2016-01-27 00:00:00
1130    2015-12-10 00:00:00
1132    2015-12-08 00:00:00
1136    2015-10-01 00:00:00
Name: Publication date, Length: 138, dtype: object

In [18]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [19]:
cost_df['Cost (inflation-adjusted)'].dropna()

36      1.914000e+08
130     2.581632e+07
169     3.931897e+06
258     7.835203e+07
274     1.503763e+06
344     1.252379e+07
370     1.635573e+06
391     1.347019e+07
434     1.493975e+06
435     8.150695e+05
443     1.238906e+07
465     1.319586e+06
467     3.108410e+05
490     1.930600e+05
494     2.093016e+06
498     3.499808e+06
537     6.405653e+06
544     9.869716e+05
574     1.655001e+05
632     2.513914e+05
647     6.197414e+05
710     9.691116e+05
719     4.324883e+06
759     6.971103e+05
786     3.889027e+05
790     2.366316e+05
804     8.817783e+05
805     2.160704e+05
830     1.600176e+05
880     6.387439e+02
913     3.287635e+03
916     1.164818e+04
927     4.434402e+03
979     1.381309e+02
1036    3.344073e+04
1090    1.947392e+05
Name: Cost (inflation-adjusted), dtype: float64

In [20]:
cost_df['Cost (inflation-adjusted)'].notna().sum()

36

# Regression

In [22]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [23]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     50.67
Date:                Wed, 06 Mar 2024   Prob (F-statistic):           3.16e-08
Time:                        17:44:53   Log-Likelihood:                -44.077
No. Observations:                  36   AIC:                             92.15
Df Residuals:                      34   BIC:                             95.32
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1113.7155    157.258     -7.082      0.000   -1433.302    -794.129
x1             0.5539      0.078      7.119      0.000       0.396       0.712
==============================================================================
Omnibus:                        1.388   Durbin-Watson:                   1.376
Prob(Omnibus):                  0.500   Jarque-Bera (JB):                0.705
Skew:                          -0.328   Prob(JB):                        0.703
Kurtosis:                       3.198   Cond. No.                     2.25e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.25e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
print_growth_rates(reg_results)

0.55 OOMs/year (95% CI: 0.40, 0.71)
3.6x/year (95% CI: 2.5x, 5.2x)
doubling time of 7 months (95% CI: 5, 9)


In [25]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [26]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,2.410386,0.489386,1.415833,3.404939,0.422341,4.398431,2015.00000,2015-01-01
1,2.466336,0.481866,1.487066,3.445606,0.485893,4.446780,2015.10101,2015-02-06
2,2.522287,0.474357,1.558277,3.486296,0.549345,4.495229,2015.20202,2015-03-15
3,2.578237,0.466859,1.629465,3.527009,0.612695,4.543779,2015.30303,2015-04-21
4,2.634187,0.459374,1.700628,3.567747,0.675943,4.592431,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,7.725671,0.311872,7.091871,8.359471,5.891307,9.560034,2024.59596,2024-08-06
96,7.781621,0.318900,7.133538,8.429704,5.942274,9.620968,2024.69697,2024-09-12
97,7.837572,0.325966,7.175128,8.500015,5.993116,9.682028,2024.79798,2024-10-19
98,7.893522,0.333068,7.216646,8.570398,6.043833,9.743210,2024.89899,2024-11-25


# Plots

In [27]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [32]:
label_systems = ['GNMT', 'Megatron-BERT', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra']

fig = px.scatter(
    cost_df.loc[cost_df['System'].isin(label_systems)],
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)

fig.add_scatter(
    x=cost_df['Publication date'],
    y=cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean ({10**reg_results.params[1]:.1f}x per year)',
)
# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)

fig.update_traces(textposition='top center')

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=0.62,
))

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=100, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

# Export data

In [29]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Open-source',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Open-source,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Training time (chip hours),Cost,Cost (inflation-adjusted)
36,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",NaN,Gemini: A Family of Highly Capable Multimodal ...,2023-12-06 00:00:00,Google DeepMind,NaN,5.000000e+25,NaN,...,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,132000000.0,191400000.0,1.914000e+08
43,Qwen-72B,Language,"Chat,Code generation",Permissive license,NaN,2023-11-30 00:00:00,Alibaba,7.200000e+10,1.300000e+24,NaN,...,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Inflection-2,Language,Language modelling,API accessible,Inflection-2: The Next Step Up,2023-11-22 00:00:00,Inflection AI,NaN,1.001000e+25,NaN,...,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN,NaN
55,Nemotron-3-8B,Language,"Chat,Language Generation",Permissive license,NVIDIA AI Foundation Models: Build Custom Ente...,2023-11-15 00:00:00,NVIDIA,8.000000e+09,1.800000e+23,NaN,...,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,CogVLM,"Multimodal,Vision,Language","Image captioning,Visual question answering,Chat",Permissive license,CogVLM: Visual Expert for Pretrained Language ...,2023-11-06 00:00:00,"Tsinghua University,Zhipu AI,Beihang University",1.700000e+10,1.988064e+22,NaN,...,"China,China,China",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,GNMT,Language,Translation,NaN,Google's Neural Machine Translation System: Br...,2016-09-26 00:00:00,Google,2.780000e+08,6.900000e+21,360000000.0,...,Multinational,NaN,NaN,96.0,NaN,NaN,NaN,414720.0,178329.6,1.947392e+05
1126,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27 00:00:00,DeepMind,NaN,1.900000e+21,29400000.0,...,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1130,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10 00:00:00,Microsoft,6.000000e+07,1.210000e+19,1280000.0,...,Multinational,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08 00:00:00,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,163339200.0,...,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)